In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json
from datasets import load_dataset

# 최적화된 모델 로드
model_path = "./llama-3.1-korean-8b-hf-stable/checkpoint-45"
model = AutoModelForCausalLM.from_pretrained(
    model_path, 
    torch_dtype=torch.bfloat16,
    device_map="auto",           # 자동 GPU 배치
    low_cpu_mem_usage=True,      # CPU 메모리 절약
    use_cache=True              # 추론 캐시 활성화
)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# PyTorch 컴파일 최적화 (20-30% 속도 향상)
model = torch.compile(model)

# Test 데이터셋 로드
test_dataset = load_dataset("json", data_files="../data/test_dataset.json", split="train")
print(f"Test 데이터 개수: {len(test_dataset)}")

# 몇 개 샘플 평가
# for i in range(min(3, len(test_dataset))):  # 처음 3개만
sample = test_dataset[3]
messages = sample['messages']

# 시스템 + 유저 프롬프트만 사용
test_messages = [
    messages[0],  # system
    messages[1]   # user
]

# 정답 (기대 출력)
expected_output = messages[2]['content']

# 모델 추론
formatted_prompt = tokenizer.apply_chat_template(
    test_messages, 
    tokenize=False, 
    add_generation_prompt=True
)

# 입력을 GPU로 보내기 (중요!)
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=8192,            # 모델 한계까지
        min_new_tokens=4000,            # 충분한 최소값
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=None,              # EOS 완전 무시
        use_cache=False,                # 캐시 끄고 완전 생성
        early_stopping=False
    )

generated_output = tokenizer.decode(
    outputs[0][inputs['input_ids'].shape[1]:], 
    skip_special_tokens=True
)

print(f"\n{'='*60}")
print(f"테스트 케이스 {i+1}:")
print(f"{'='*60}")
print(f"입력: {messages[1]['content'][:100]}...")
print(f"\n기대 출력 (처음 200자):")
print(f"{expected_output[:200]}...")
print(f"\n실제 출력 (처음 200자):")
print(f"{generated_output[:200]}...")

# 간단한 품질 평가
required_keywords = ["project_summary", "relationships_data", "ERD_data", "API_specs_data"]
found_keywords = sum(1 for kw in required_keywords if kw in generated_output)

if found_keywords >= 3:  # 4개 중 3개 이상
    print(f"\n✅ 구조적 일관성: 좋음 ({found_keywords}/4 키워드 포함)")
else:
    print(f"\n❌ 구조적 일관성: 부족 ({found_keywords}/4 키워드 포함)")
    
print(f"\n출력 길이: {len(generated_output)}자")
print("-" * 60)

print("\n🎯 평가 완료!")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Test 데이터 개수: 10


In [13]:
generated_output

" **프로젝트 상세 정보:**\n{'project_summary': {'title': '자율주행 기반 해변 쓰레기 수거 RC카', 'category': '공학경진대회 프로젝트', 'target_users': ['공학자', '엔지니어', '환경 보호에 관심 있는 개인'],'main_purpose': '자율주행 기술을 접목해 쓰레기 수거 기능을 통해 환경 문제 해결', 'key_features': [{'feature': '자율주행', 'description': 'Keras 기반 딥러닝 모델을 통해 경로를 학습'}, {'feature': '쓰레기 수거', 'description': 'OpenCV로 인식된 쓰레기 객체를 수거'}, {'feature': '임베디드 시스템', 'description': '라즈베리파이 기반으로 시스템을 구축'}, {'feature': '카메라 영상 처리', 'description': 'OpenCV로 도로 및 쓰레기 객체 인식'}, {'feature': '행동 학습', 'description': 'Behavioral Cloning을 통해 자율 주행 경로 학습'}], 'core_technologies': [{'category': 'Programming', 'technologies': ['Python']}, {'category': 'Machine Learning', 'technologies': ['Keras', '딥러닝']}, {'category': 'Computer Vision', 'technologies': ['OpenCV']}, {'category': 'Embedded System', 'technologies': ['Embedded Linux', '라즈베리파이']}, {'category': 'Behavioral Learning', 'technologies': ['Behavioral Cloning']}], 'problem_solving': {'current_problem': '환경 문제 해결을 위한 쓰레기 수거의 비효율성과 자율주행 기술의